In [76]:
import pandas as pd 
import os 
import httpx
import asyncio
from tqdm.asyncio import tqdm

In [77]:
table = pd.read_csv("./table/train_all_data_new.csv")
table

,date,title,author,url,image_link,hotNumber,is_hot,image_state
0,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/BdmZ7Ps.jpg,71,1,True
1,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/bBiw4IS.jpg,71,1,True
2,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/KDmRgeg.jpg,71,1,True
3,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/hRHpy8X.jpg,71,1,True
4,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/P5rI8UM.jpg,71,1,True
...,...,...,...,...,...,...,...,...
67025,2023-Dec-31-21:57:42,[正妹] 潤娥,wafiea708,https://www.ptt.cc/bbs/Beauty/M.1704031064.A.D...,https://i.imgur.com/t0YhewP.gif,38,1,False
67026,2023-Dec-31-21:57:42,[正妹] 潤娥,wafiea708,https://www.ptt.cc/bbs/Beauty/M.1704031064.A.D...,https://i.imgur.com/B3SB2WY.jpg,38,1,True
67027,2023-Dec-31-21:57:42,[正妹] 潤娥,wafiea708,https://www.ptt.cc/bbs/Beauty/M.1704031064.A.D...,https://i.imgur.com/rCnbrGX.jpg,38,1,True
67028,2023-Dec-31-21:57:42,[正妹] 潤娥,wafiea708,https://www.ptt.cc/bbs/Beauty/M.1704031064.A.D...,https://i.imgur.com/zGIk6WD.jpg,38,1,True


In [78]:
from CrawlerHW import CrawlerHW
from CrawlerData import CrawlerData
import cv2

In [79]:
image_link_list = table["image_link"]


In [80]:
IMAGE_FOLDER = "./database"

async def fetch_image(url: str , client:httpx.AsyncClient):
    file_name = url.replace("https://" , "").replace("http://" , "").replace("/","-")
    file_path = os.path.join(IMAGE_FOLDER , file_name)
    
    # file is in
    if os.path.exists(file_path):
       return None 
    
    await asyncio.sleep(CrawlerHW.get_random_wait_time())
    
    try:
        image_get = await client.get(url , headers=CrawlerHW.get_header())
    except Exception as e:
        return {"url" : url , "error": str(e)}
    
    # not get image
    if image_get.status_code != 200:
        return {"url" : url , "status_code": image_get.status_code}
    
    # get image 
    try:
        image = CrawlerData.bytes_to_image(image_get.content)
        cv2.imwrite(file_path , image)
    except Exception as e:
        return {"url" : url , "error" : [str(e) , "image_get.content"]}
    
    return None
    

In [81]:
async def gather(*task):
    "make like asyncio"

    ## cut to chunk
    chunk_task = [
        task[i : i + CrawlerHW.CHUNK_SIZE]
        for i in range(0, len(task), CrawlerHW.CHUNK_SIZE)
    ]

    tasks_result = [await tqdm.gather(*chunk_pack , desc=f"chunk: {index}" ,unit="image") for index , chunk_pack in enumerate(chunk_task)]
    # add to result
    tasks_result = sum(tasks_result, [])
    return tasks_result

In [82]:
async with httpx.AsyncClient() as client:
    tasks = [fetch_image(url , client) for url in image_link_list]
    
    error_result = await gather(*tasks)

chunk: 670: 100%|██████████| 30/30 [00:07<00:00,  4.09image/s]


In [83]:
print(error_result)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, {'url': 'https://i.imgur.com/XaKrUlM.jpg', 'status_code': 302}, {'url': 'https://i.imgur.com/uXh36LW.png', 'status_code': 302}, {'url': 'https://i.imgur.com/IdWjvsc.png', 'status_code': 302}, {'url': 'https://i.imgur.com/0gAyDJP.png', 'status_code': 302}, {'url': 'https://i.imgur.com/YOO1vKw.png', 'status_code': 302}, {'url': 'https://i.imgur.com/sIFeFNb.png', 'status_code': 302}, {'url': 'https://i.imgur.com/s21w4ut.png', 'status_code': 302}, {'url': 'https://i.imgur.com/LR2UOd9.png', 'status_code': 302}, {'url': 'https://i.imgur.com/LhlGvVh.png', 'status_code': 302}, {'url': 'https://i.imgur.com/s8RHG0a.png', 'status_code': 302}, {'url': 'https://i.imgur.com/zGcqYVz.png', 'status_code': 302}, {'url': 'https://i.imgur.com/m8bZ5kD.png', 'status_code': 302}, {'url': 'https://i.imgur.com/Mtx8H3Q.png', 'status_code': 302}, {'url': 'https://i.imgur.com/o5uM54v.png', 'status_code': 3

In [85]:
error_result = [item for item in error_result if item is not None]
error_result

[{'url': 'https://i.imgur.com/XaKrUlM.jpg', 'status_code': 302},
 {'url': 'https://i.imgur.com/uXh36LW.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/IdWjvsc.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/0gAyDJP.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/YOO1vKw.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/sIFeFNb.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/s21w4ut.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/LR2UOd9.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/LhlGvVh.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/s8RHG0a.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/zGcqYVz.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/m8bZ5kD.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/Mtx8H3Q.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/o5uM54v.png', 'status_code': 302},
 {'url': 'https://i.imgur.com/zUwGWOn.png', 'status_code': 302},
 {'url': 'https://i.imgur

In [86]:
error_table = pd.DataFrame(error_result)

In [87]:
error_table

,url,status_code,error
0,https://i.imgur.com/XaKrUlM.jpg,302.0,NaN
1,https://i.imgur.com/uXh36LW.png,302.0,NaN
2,https://i.imgur.com/IdWjvsc.png,302.0,NaN
3,https://i.imgur.com/0gAyDJP.png,302.0,NaN
4,https://i.imgur.com/YOO1vKw.png,302.0,NaN
...,...,...,...
10368,https://i.imgur.com/7w4BAx0.gif,NaN,[OpenCV(4.9.0) D:\a\opencv-python\opencv-pytho...
10369,https://i.imgur.com/3kTOKxQ.gif,NaN,[OpenCV(4.9.0) D:\a\opencv-python\opencv-pytho...
10370,https://i.imgur.com/CO0JYEw.gif,NaN,[OpenCV(4.9.0) D:\a\opencv-python\opencv-pytho...
10371,https://i.imgur.com/t0YhewP.gif,NaN,[OpenCV(4.9.0) D:\a\opencv-python\opencv-pytho...


In [89]:
error_table.to_csv("./table/error_table_new.csv", index=False)

In [94]:
table = table.drop_duplicates()

In [100]:
except_len = len(table) - len(error_table)
except_len

56631

In [101]:
acc_len = len(os.listdir("./database"))
acc_len

55597

In [102]:
except_len - acc_len

1034